# Constructive Heuristic Template
# Type here the Name of the Combinatorial problem you studied
Prof. María Angélica Salazar Aguilar

Selected Topics of Optimization

---
This notebook is a template for documenting a combinatorial optimization problem and its constructive heuristic. Follow the instructions in each section and provide the required content.



## 1. Team: TeamID

Provide the names, student IDs, and contact information of all team members below.

|Student ID      | Name  | Email             |
|----------------|------------|-------------------|
|                |            |                   |
|                |            |                   |
|                |            |                   |

## 2. General Description of the Problem

Write a brief overview of the combinatorial problem being studied, including its relevance and applications.

## 3. Mathematical Formulation

Present the mathematical formulation of the problem, including variables, constraints, and objective function. Use LaTeX for equations, for example:

Let $x$ be a decision variable, $C$ the cost matrix, and $S$ the set of feasible solutions.

$$
\text{Minimize} \quad f(x) = \sum_{i \in S} C_i x_i
$$

Subject to:
$$
\sum_{i \in S} x_i = 1
$$
$$
 x_i \in \{0,1\} \quad \forall i \in S
$$

## 4. Pseudocode of the Proposed Constructive Heuristic

Write the pseudocode for the constructive heuristic proposed to solve the problem. Use code formatting for clarity.

```
Input: [Describe input]
Initialize solution
While [termination condition not met]:
    Select next element according to heuristic
    Update solution
Output: [Describe output]
```

## 5. Description and Definition of Main Functions

Describe the main functions used in the heuristic, including their purpose, inputs, and outputs. For example:

- `load_data(file_path)`: Loads problem data from a file.
- `construct_solution(data)`: Builds a solution using the heuristic.
- `evaluate_solution(solution)`: Computes the objective value of a solution.
- `visualize_solution(solution)`: Plots the solution.

Add more functions as needed and explain their roles.

In [52]:
# 6. Implementation of the Constructive Heuristic
# Implement your constructive heuristic below using Python.

def load_data(file_path):
    # TODO: Implement data loading
    pass

def construct_solution(data):
    # TODO: Implement heuristic construction
    pass

def evaluate_solution(solution):
    # TODO: Implement solution evaluation
    pass

def visualize_solution(solution):
    # TODO: Implement solution visualization
    pass


In [53]:
## 7. Main Function to Run the Code
# Write a main function that runs the heuristic on sample instances.

def main():
    file_path = 'your_data_file.csv'  # Update with your data file
    data = load_data(file_path)
    solution = construct_solution(data)
    value = evaluate_solution(solution)
    print('Solution:', solution)
    print('Objective value:', value)
    visualize_solution(solution)

# Uncomment to run
# main()

## 8. Computational Results and Discussion
Provide the url where you downloaded the  tested instances and mention the main characteristics of the ones you tested.

Include a table summarizing the results of at least 15 different tested instances. Discuss the performance and findings below.

| Instance | Objective Value Heuristic | Objective Value (Optimal) | Gap (%) |
|----------|----------|----------------|----------|
|          |          |                |          |
|          |          |                |          |

**Discussion:**
- Summarize the effectiveness of your heuristic.
- Compare results and computation time across instances.
- Discuss any patterns or insights observed.


## 9. General Conclusions

Summarize the main findings, lessons learned, and possible future work.

## 10. Revised References

List all references and sources used in your project. Format them according to a standard citation style (e.g., APA, IEEE).

- Author, Title, Journal/Conference, Year.
- Book, Author, Year.
- Website, URL, Access Date.